## Import packages

In [1]:
import os
import json
import shutil
import numpy as np
from sklearn.cluster import DBSCAN

from utils import TopKKeeper

## Alignability of embeddings

### Load embeddings

In [121]:
def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def save_json(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

def get_model_info_from_file_name(filename):
    model_code = input_name.split('-')[1]
    model_name = '-'.join(model_code.split('_')[:-1]).replace('inception-v3', 'inception_v3')
    epoch = int(model_code.split('_')[-1])
    return model_name, epoch

def eu_dist(a, b):
    d = np.sum((a - b) ** 2)
    return d

def cossim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def save_ex_patches(model_name, neuron_id, ex_patch_dir, output_dir):
    
    # Input img directory
    input_dir = os.path.join(ex_patch_dir, model_name, 'data')
    
    # Img names
    img_name_lst = []
    for img_name in os.listdir(input_dir):
        if neuron_id + '-' in img_name:
            img_name_lst.append(img_name)
            
    # Copy images
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for img_i, img_name in enumerate(img_name_lst):
        img_path = os.path.join(input_dir, img_name)
        shutil.copy2(img_path, os.path.join(output_dir, 'img-{}.jpg'.format(img_i)))
        
def mkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [44]:
input_dir = '../../data/embedding/emb-3-base-vgg16-207'
ex_patch_dir = '../../data/neuron_feature'

models = [
    'vgg16-512-0.01-0.9',
    'inception_v3-512-0.5-0.9'
]

emb = {}
for input_name in os.listdir(input_dir):
    
    # Ignore unnecessary models
    if 'pretrained' in input_name:
        continue
    model_name, epoch = get_model_info_from_file_name(input_name)
    if model_name not in models:
        continue
        
    # Read embedding data
    input_path = os.path.join(input_dir, input_name)
    data = load_json(input_path)
    
    # Save embedding data
    emb['{}-{}'.format(model_name, epoch)] = data

In [129]:
list(emb.keys())

['inception_v3-512-0.5-0.9-11',
 'vgg16-512-0.01-0.9-207',
 'vgg16-512-0.01-0.9-21',
 'inception_v3-512-0.5-0.9-3',
 'inception_v3-512-0.5-0.9-121',
 'vgg16-512-0.01-0.9-5']

### 1. Base model's concept alignability

In [187]:
base_model_name = 'vgg16-512-0.01-0.9-207'
base_model_emb = emb[base_model_name]
base_model_neurons = list(base_model_emb.keys())

num_neurons_by_layer = {}
for neuron in base_model_neurons:
    layer = neuron.split('-')[0]
    layer_idx = int(layer.split('_')[-1])
    if layer_idx < 10:
        continue
    if layer not in num_neurons_by_layer:
        num_neurons_by_layer[layer] = 0
    num_neurons_by_layer[layer] += 1
    
num_neurons_by_layer

{'Sequential_0_Conv2d_10': 256,
 'Sequential_0_Conv2d_12': 256,
 'Sequential_0_Conv2d_14': 256,
 'Sequential_0_Conv2d_17': 512,
 'Sequential_0_Conv2d_19': 512,
 'Sequential_0_Conv2d_21': 512,
 'Sequential_0_Conv2d_24': 512,
 'Sequential_0_Conv2d_26': 512,
 'Sequential_0_Conv2d_28': 512}

In [188]:
layers = list(num_neurons_by_layer.keys())
probs = np.array(list(num_neurons_by_layer.values()))
probs = probs / np.sum(probs)
sampled_layers = np.random.choice(layers, 15, p=probs)
sampled_layers

array(['Sequential_0_Conv2d_19', 'Sequential_0_Conv2d_28',
       'Sequential_0_Conv2d_12', 'Sequential_0_Conv2d_19',
       'Sequential_0_Conv2d_17', 'Sequential_0_Conv2d_28',
       'Sequential_0_Conv2d_10', 'Sequential_0_Conv2d_21',
       'Sequential_0_Conv2d_21', 'Sequential_0_Conv2d_28',
       'Sequential_0_Conv2d_19', 'Sequential_0_Conv2d_14',
       'Sequential_0_Conv2d_17', 'Sequential_0_Conv2d_19',
       'Sequential_0_Conv2d_19'], dtype='<U22')

In [189]:
sampled_neurons = []
for sampled_layer in sampled_layers:
    while True:
        neuron_idx = np.random.choice(num_neurons_by_layer[sampled_layer])
        neuron_id = '{}-{}'.format(sampled_layer, neuron_idx)
        if neuron_id not in sampled_neurons:
            sampled_neurons.append(neuron_id)
            break
sorted(sampled_neurons, key=lambda x: int(x.split('-')[0].split('_')[-1]))

['Sequential_0_Conv2d_10-17',
 'Sequential_0_Conv2d_12-228',
 'Sequential_0_Conv2d_14-71',
 'Sequential_0_Conv2d_17-172',
 'Sequential_0_Conv2d_17-412',
 'Sequential_0_Conv2d_19-22',
 'Sequential_0_Conv2d_19-401',
 'Sequential_0_Conv2d_19-25',
 'Sequential_0_Conv2d_19-47',
 'Sequential_0_Conv2d_19-303',
 'Sequential_0_Conv2d_21-326',
 'Sequential_0_Conv2d_21-397',
 'Sequential_0_Conv2d_28-11',
 'Sequential_0_Conv2d_28-267',
 'Sequential_0_Conv2d_28-112']

In [190]:
# Find sets
parent_dir = '../../data/human-experiment/alignability-example/NeuEvo-base_model'
for set_idx, sampled_neuron in enumerate(sampled_neurons):
    
    # Output dir
    output_dir = os.path.join(parent_dir, 'set-{}'.format(set_idx))
    mkdir(output_dir)
    img_dir = os.path.join(output_dir, 'img')
    mkdir(img_dir)
    
    # Find 5 closest neurons
    target_emb = np.array(base_model_emb[sampled_neuron])
    cluster = TopKKeeper(5)
    for neuron in base_model_emb:
        neuron_emb = np.array(base_model_emb[neuron])
        cos = cossim(neuron_emb, target_emb)
        d = -eu_dist(neuron_emb, target_emb)
        cluster.insert(cos, key=neuron)
    neuron_set = cluster.keys
    
    # Choose a random neuron
    while True:
        rand_neuron = np.random.choice(base_model_neurons)
        if rand_neuron not in neuron_set:
            break
    neuron_set.append(rand_neuron)
    np.random.shuffle(neuron_set)
    rand_idx = neuron_set.index(rand_neuron)
    
    # Make neuron info and save it
    neurons = {}
    for i, neuron in enumerate(neuron_set):
        neurons['neuron-{}'.format(i)] = {
            'model': base_model_name,
            'neuron': neuron,
            'random': 'yes' if i == rand_idx else 'no'
        }
    path = os.path.join(output_dir, 'neuron.json')
    save_json(neurons, path)

    # Save images
    for neuron_idx, neuron in enumerate(neuron_set):
        final_dir = os.path.join(img_dir, 'neuron-{}'.format(neuron_idx))
        mkdir(final_dir)
        save_ex_patches(base_model_name, neuron, ex_patch_dir, final_dir)

### 2. Random alignability 

In [191]:
parent_dir = '../../data/human-experiment/alignability-example/Random'
for set_idx in range(20):
    
    # Output dir
    output_dir = os.path.join(parent_dir, 'set-{}'.format(set_idx))
    mkdir(output_dir)
    img_dir = os.path.join(output_dir, 'img')
    mkdir(img_dir)
    
    # Sample neurons
    neuron_set = np.random.choice(base_model_neurons, 6, replace=False)
    
    # Make neuron info and save it
    neurons = {}
    for i, neuron in enumerate(neuron_set):
        neurons['neuron-{}'.format(i)] = {
            'model': base_model_name,
            'neuron': neuron,
            'random': 'yes'
        }
    path = os.path.join(output_dir, 'neuron.json')
    save_json(neurons, path)
    
    # Save images
    for neuron_idx, neuron in enumerate(neuron_set):
        final_dir = os.path.join(img_dir, 'neuron-{}'.format(neuron_idx))
        mkdir(final_dir)
        save_ex_patches(base_model_name, neuron, ex_patch_dir, final_dir)

### 3. Same model, different epoch

In [192]:
models

['vgg16-512-0.01-0.9', 'inception_v3-512-0.5-0.9']

In [193]:
list(emb.keys())

['inception_v3-512-0.5-0.9-11',
 'vgg16-512-0.01-0.9-207',
 'vgg16-512-0.01-0.9-21',
 'inception_v3-512-0.5-0.9-3',
 'inception_v3-512-0.5-0.9-121',
 'vgg16-512-0.01-0.9-5']

In [194]:
model_to_epochs = {
    'vgg16-512-0.01-0.9': [5, 21, 207],
    'inception_v3-512-0.5-0.9': [3, 11, 121],
}

In [195]:
num_ex_per_model = 16
parent_dir = '../../data/human-experiment/alignability-example/NeuEvo-same_model-diff_epoch'

for model_idx, model_name in enumerate([models[0]]):
    
    # Ready
    epochs = model_to_epochs[model_name]
    model_neurons = list(emb['{}-{}'.format(model_name, epochs[0])].keys())
    num_neurons_by_layer = {}
    for neuron in model_neurons:
        layer = neuron.split('-')[0]
        layer_idx = int(layer.split('_')[-1])
        if layer_idx < 10:
            continue
        if layer not in num_neurons_by_layer:
            num_neurons_by_layer[layer] = 0
        num_neurons_by_layer[layer] += 1
    layers = list(num_neurons_by_layer.keys())
    probs = np.array(list(num_neurons_by_layer.values()))
    probs = probs / np.sum(probs)
    
    # Embedding of all epochs
    model_emb = {}
    for epoch in epochs:
        model_id = '{}-{}'.format(model_name, epoch)
        for neuron in emb[model_id]:
            neuron_id = '{}-{}'.format(epoch, neuron)
            model_emb[neuron_id] = emb[model_id][neuron]
    
    # Generate sets for each model
    for ex_idx in range(num_ex_per_model):
        
        # Output dir
        set_idx = num_ex_per_model * model_idx + ex_idx
        output_dir = os.path.join(parent_dir, 'set-{}'.format(set_idx))
        mkdir(output_dir)
        img_dir = os.path.join(output_dir, 'img')
        mkdir(img_dir)
        
        # Sample neuron in the last epoch
        sampled_layer = np.random.choice(layers, p=probs)
        sampled_neuron_idx = np.random.choice(num_neurons_by_layer[sampled_layer])
        sampled_neuron_id = '{}-{}'.format(sampled_layer, sampled_neuron_idx)
        sampled_neuron_epoch = epochs[-1]
        sampled_neuron_emb = model_emb['{}-{}'.format(sampled_neuron_epoch, sampled_neuron_id)]
        
        # Find 5 closest neurons
        target_emb = sampled_neuron_emb
        cluster = TopKKeeper(5)
        for neuron in model_emb:
            neuron_emb = np.array(model_emb[neuron])
            cos = cossim(neuron_emb, target_emb)
            d = -eu_dist(neuron_emb, target_emb)
            cluster.insert(cos, key=neuron)
        neuron_set = cluster.keys
        
        # Choose a random neuron
        all_neurons = list(model_emb.keys())
        while True:
            rand_neuron = np.random.choice(all_neurons)
            if rand_neuron not in neuron_set:
                break
        neuron_set.append(rand_neuron)
        np.random.shuffle(neuron_set)
        rand_idx = neuron_set.index(rand_neuron)
        
        # Make neuron info and save it
        neurons = {}
        for i, neuron in enumerate(neuron_set):
            neuron_model_epoch = int(neuron.split('-')[0])
            neuron_id = '-'.join(neuron.split('-')[1:])
            neuron_model = '{}-{}'.format(model_name, neuron_model_epoch)
            neurons['neuron-{}'.format(i)] = {
                'model': neuron_model,
                'neuron': neuron_id,
                'random': 'yes' if i == rand_idx else 'no'
            }
        path = os.path.join(output_dir, 'neuron.json')
        save_json(neurons, path)
        
        # Save images
        for neuron_idx, neuron in enumerate(neuron_set):
            neuron_model_epoch = int(neuron.split('-')[0])
            neuron_model = '{}-{}'.format(model_name, neuron_model_epoch)
            neuron = '-'.join(neuron.split('-')[1:])
            final_dir = os.path.join(img_dir, 'neuron-{}'.format(neuron_idx))
            mkdir(final_dir)
            save_ex_patches(neuron_model, neuron, ex_patch_dir, final_dir)

### 4. Different models, different epochs 

In [196]:
parent_dir = '../../data/human-experiment/alignability-example/NeuEvo-diff_model-diff_epoch'

In [197]:
models

['vgg16-512-0.01-0.9', 'inception_v3-512-0.5-0.9']

In [198]:
list(emb.keys())

['inception_v3-512-0.5-0.9-11',
 'vgg16-512-0.01-0.9-207',
 'vgg16-512-0.01-0.9-21',
 'inception_v3-512-0.5-0.9-3',
 'inception_v3-512-0.5-0.9-121',
 'vgg16-512-0.01-0.9-5']

In [199]:
model_to_epochs = {
    'vgg16-512-0.01-0.9': [5, 21, 207],
    'inception_v3-512-0.5-0.9': [3, 11, 121],
}

In [200]:
base_model_name = 'vgg16-512-0.01-0.9-207'
base_model_emb = emb[base_model_name]
base_model_neurons = list(base_model_emb.keys())

num_neurons_by_layer = {}
for neuron in base_model_neurons:
    layer = neuron.split('-')[0]
    layer_idx = int(layer.split('_')[-1])
    if layer_idx < 10:
        continue
    if layer not in num_neurons_by_layer:
        num_neurons_by_layer[layer] = 0
    num_neurons_by_layer[layer] += 1
    
num_neurons_by_layer

{'Sequential_0_Conv2d_10': 256,
 'Sequential_0_Conv2d_12': 256,
 'Sequential_0_Conv2d_14': 256,
 'Sequential_0_Conv2d_17': 512,
 'Sequential_0_Conv2d_19': 512,
 'Sequential_0_Conv2d_21': 512,
 'Sequential_0_Conv2d_24': 512,
 'Sequential_0_Conv2d_26': 512,
 'Sequential_0_Conv2d_28': 512}

In [201]:
layers = list(num_neurons_by_layer.keys())
probs = np.array(list(num_neurons_by_layer.values()))
probs = probs / np.sum(probs)

In [203]:
for set_idx in range(16):
    
    # Output dir
    output_dir = os.path.join(parent_dir, 'set-{}'.format(set_idx))
    mkdir(output_dir)
    img_dir = os.path.join(output_dir, 'img')
    mkdir(img_dir)

    # Ready
    all_embs = {}
    for model in emb:
        for neuron in emb[model]:
            neuron_id = '{}___{}'.format(model, neuron)
            all_embs[neuron_id] = emb[model][neuron]
    all_neurons = list(all_embs.keys())
    
    # Sample neuron
    sampled_layer = np.random.choice(layers, p=probs)
    sampled_neuron_idx = np.random.choice(num_neurons_by_layer[sampled_layer])
    sampled_neuron_id = '{}-{}'.format(sampled_layer, sampled_neuron_idx)
    sampled_neuron_emb = all_embs['{}___{}'.format(base_model_name, sampled_neuron_id)]
            
    # Find 5 closest neurons
    target_emb = sampled_neuron_emb
    cluster = TopKKeeper(5)
    for neuron in all_embs:
        neuron_emb = np.array(all_embs[neuron])
        cos = cossim(neuron_emb, target_emb)
        d = -eu_dist(neuron_emb, target_emb)
        cluster.insert(cos, key=neuron)
    neuron_set = cluster.keys
        
    # Choose a random neuron
    all_neurons = list(all_embs.keys())
    while True:
        rand_neuron = np.random.choice(all_neurons)
        if rand_neuron not in neuron_set:
            break
    neuron_set.append(rand_neuron)
    np.random.shuffle(neuron_set)
    rand_idx = neuron_set.index(rand_neuron)
        
    # Make neuron info and save it
    neurons = {}
    for i, neuron in enumerate(neuron_set):
        model_name = neuron.split('___')[0]
        neuron_id = neuron.split('___')[1]
        neurons['neuron-{}'.format(i)] = {
            'model': model_name,
            'neuron': neuron_id,
            'random': 'yes' if i == rand_idx else 'no'
        }
    path = os.path.join(output_dir, 'neuron.json')
    save_json(neurons, path)
        
    # Save images
    for neuron_idx, neuron in enumerate(neuron_set):
        model_name = neuron.split('___')[0]
        neuron_id = neuron.split('___')[1]
        final_dir = os.path.join(img_dir, 'neuron-{}'.format(neuron_idx))
        mkdir(final_dir)
        save_ex_patches(model_name, neuron_id, ex_patch_dir, final_dir)

### 5. Hand-picked

In [257]:
# hand_picked = {
#     'cyan': ['BasicConv2d_0-0', 'BasicConv2d_0-2', 'BasicConv2d_0-17', 'InceptionA_9-19', 'InceptionB_10-760'],
#     'blue': ['BasicConv2d_0-4', 'BasicConv2d_0-14', 'BasicConv2d_1-20', 'InceptionA_7-131', 'InceptionA_9-280'],
#     'bright-red-yellow': ['BasicConv2d_0-30', 'BasicConv2d_2-3', 'InceptionA_8-248', 'InceptionA_9-226', 'InceptionB_10-706'],
#     'cross': ['BasicConv2d_5-54', 'BasicConv2d_5-125', 'InceptionA_7-21', 'InceptionA_7-52', 'InceptionA_7-60'],
#     'green': ['InceptionA_7-113', 'InceptionA_7-251', 'InceptionA_8-27', 'InceptionA_8-228', 'InceptionA_8-236'],
#     'corner': ['InceptionA_7-194', 'InceptionA_7-204', 'InceptionA_7-217', 'InceptionA_8-60', 'InceptionA_8-121'],
#     'eyes': ['InceptionB_10-192', 'InceptionB_10-251', 'InceptionB_10-576', 'InceptionC_11-55', 'InceptionC_11-201'],
#     'letter': ['InceptionB_10-241', 'InceptionB_10-491', 'InceptionB_10-707', 'InceptionC_11-68', 'InceptionC_11-411'],
#     'nose': ['InceptionB_10-478', 'InceptionC_11-263', 'InceptionC_11-623', 'InceptionC_11-638', 'InceptionC_12-136-6'],
#     'human': ['InceptionC_11-195', 'InceptionC_11-249-7', 'InceptionC_11-680', 'InceptionC_12-110', 'InceptionC_12-325'],
#     'diagonal': ['InceptionC_11-578', 'InceptionC_11-614', 'InceptionC_11-632', 'InceptionC_11-641', 'InceptionC_11-646'],
#     'spreading': ['InceptionC_11-262', 'InceptionC_11-593', 'InceptionC_12-47', 'InceptionC_12-264', 'InceptionC_12-279'],
#     'stripe-check': ['InceptionC_11-605', 'InceptionC_11-651', 'InceptionC_11-677', 'InceptionC_11-706', 'InceptionC_11-731'],
#     'hair': ['InceptionC_12-217', 'InceptionC_12-211', 'InceptionC_12-249', 'InceptionC_12-359', 'InceptionC_12-361'],
#     'head': ['InceptionC_12-301', 'InceptionC_12-305', 'InceptionC_12-313', 'InceptionC_12-327', 'InceptionC_12-388'],
#     'red': ['InceptionC_12-416', 'InceptionC_12-424', 'InceptionC_12-432', 'InceptionC_12-441', 'InceptionC_12-447'],
#     'circle': ['InceptionC_11-568', 'InceptionC_11-732', 'InceptionC_11-750', 'InceptionC_12-645', 'InceptionC_12-661'],
#     'roof': ['InceptionC_11-288', 'InceptionC_12-276', 'InceptionC_12-493', 'InceptionC_12-658', 'InceptionC_12-731'],
#     'dog': ['InceptionC_12-506', 'InceptionC_12-525', 'InceptionC_12-529', 'InceptionC_12-744', 'InceptionC_13-264'],
#     'leaves': ['InceptionC_12-667', 'InceptionC_12-699', 'InceptionC_12-721', 'InceptionC_13-602', 'InceptionC_13-667'],
#     'sky': ['InceptionC_11-594', 'InceptionC_12-396', 'InceptionC_12-403', 'InceptionC_13-642', 'InceptionC_14-340'],
#     'watermark': ['InceptionA_7-223', 'InceptionA_8-31', 'InceptionA_8-115', 'InceptionC_13-208', 'InceptionC_13-380'],
#     'candle': ['InceptionA_7-97', 'InceptionA_7-111', 'InceptionA_8-286', 'InceptionC_14-650', 'InceptionD_16-1251'],
#     'grass': ['InceptionC_12-395', 'InceptionC_12-433', 'InceptionC_12-482', 'InceptionC_14-592', 'InceptionC_12-62'],
#     'fingers': ['InceptionC_12-619', 'InceptionC_12-669', 'InceptionC_13-491', 'InceptionC_13-561', 'InceptionC_13-738'],
#     'ball': ['InceptionE_18-535', 'InceptionE_18-537', 'InceptionE_18-567', 'InceptionE_18-612', 'InceptionE_18-640'],
#     'vertical': ['InceptionC_11-619', 'InceptionC_11-624', 'InceptionC_12-156', 'InceptionC_11-747', 'InceptionE_17-1374'],
#     'x-shape': ['BasicConv2d_5-125', 'InceptionC_11-259', 'InceptionC_11-305', 'InceptionC_12-335', 'InceptionE_17-1347'],
#     'yellow': ['BasicConv2d_0-22', 'BasicConv2d_0-28', 'InceptionA_9-160', 'InceptionA_7-119', 'InceptionE_18-1049'],
#     'frame': ['BasicConv2d_4-33', 'BasicConv2d_4-35', 'BasicConv2d_4-39', 'InceptionA_9-118', 'InceptionA_9-119'],
#     'bird': ['InceptionC_12-444', 'InceptionC_13-213', 'InceptionC_13-403', 'InceptionC_13-564', 'InceptionE_17-1967'],
#     'blue-point': ['BasicConv2d_0-4', 'BasicConv2d_0-14', 'BasicConv2d_1-20', 'InceptionC_12-504', 'InceptionC_12-562']
# }

In [258]:
model = base_model_name

In [176]:
hand_picked = {
    'animal': ["Sequential_0_Conv2d_17-214", "Sequential_0_Conv2d_17-259", "Sequential_0_Conv2d_17-272", "Sequential_0_Conv2d_17-284","Sequential_0_Conv2d_19-140"],
    'beak': ["Sequential_0_Conv2d_19-181","Sequential_0_Conv2d_19-196","Sequential_0_Conv2d_19-245","Sequential_0_Conv2d_19-427","Sequential_0_Conv2d_19-438"],
    'bird': ["Sequential_0_Conv2d_24-504","Sequential_0_Conv2d_24-310","Sequential_0_Conv2d_24-384","Sequential_0_Conv2d_24-394","Sequential_0_Conv2d_24-401"],
    'butterfly': ["Sequential_0_Conv2d_19-330","Sequential_0_Conv2d_14-217","Sequential_0_Conv2d_17-321","Sequential_0_Conv2d_19-216","Sequential_0_Conv2d_21-62"],
    'car': ["Sequential_0_Conv2d_24-237","Sequential_0_Conv2d_24-395","Sequential_0_Conv2d_26-75","Sequential_0_Conv2d_26-77","Sequential_0_Conv2d_28-324"],
    'check': ["Sequential_0_Conv2d_17-171","Sequential_0_Conv2d_17-223","Sequential_0_Conv2d_17-249","Sequential_0_Conv2d_17-262","Sequential_0_Conv2d_17-352"],
    'circles': ["Sequential_0_Conv2d_17-484","Sequential_0_Conv2d_17-498","Sequential_0_Conv2d_17-499","Sequential_0_Conv2d_17-508","Sequential_0_Conv2d_19-5"],
    'clock': ["Sequential_0_Conv2d_28-406","Sequential_0_Conv2d_28-336","Sequential_0_Conv2d_28-132","Sequential_0_Conv2d_28-87","Sequential_0_Conv2d_28-31"],
    'dog1': ["Sequential_0_Conv2d_24-505","Sequential_0_Conv2d_24-311","Sequential_0_Conv2d_24-317","Sequential_0_Conv2d_24-318","Sequential_0_Conv2d_24-324"],
    'eyes': ["Sequential_0_Conv2d_17-444","Sequential_0_Conv2d_17-459","Sequential_0_Conv2d_17-509","Sequential_0_Conv2d_17-511","Sequential_0_Conv2d_19-47"],
}
len(hand_picked)

10

In [177]:
parent_dir = '../../data/human-experiment/alignability-example/Hand-picked'

In [184]:
for i, key in enumerate(hand_picked):
    
    picked_neurons = hand_picked[key][:]
    
    # Sample neuron
    sampled_layer = np.random.choice(layers, p=probs)
    sampled_neuron_idx = np.random.choice(num_neurons_by_layer[sampled_layer])
    sampled_neuron_id = '{}-{}'.format(sampled_layer, sampled_neuron_idx)
    
    # Shuffle neurons
    picked_neurons.append(sampled_neuron_id)
    np.random.shuffle(picked_neurons)
    rand_neuron_idx = picked_neurons.index(sampled_neuron_id)
    
    # Generate output file
    dir_name = 'set-{}'.format(i)
    dir_path = os.path.join(parent_dir, dir_name)
    mkdir(dir_path)
    img_dir_path = os.path.join(dir_path, 'img')
    mkdir(img_dir_path)
        
    # Save neurons
    out_path = os.path.join(dir_path, 'neuron.json')
    neuron_output = {}
    for neuron_i, neuron_id in enumerate(picked_neurons):
        if neuron_i == rand_neuron_idx:
            neuron_output['neuron-{}'.format(neuron_i)] = {'model': model, 'neuron': neuron_id, 'random': 'yes'}
        else:
            neuron_output['neuron-{}'.format(neuron_i)] = {'model': model, 'neuron': neuron_id, 'random': 'no'}
    with open(out_path, 'w') as f:
        json.dump(neuron_output, f, indent=4)
    
    # Save example patches
    for neuron_idx, neuron in enumerate(picked_neurons):
        final_dir = os.path.join(img_dir_path, 'neuron-{}'.format(neuron_idx))
        mkdir(final_dir)
        save_ex_patches(base_model_name, neuron, ex_patch_dir, final_dir)

In [183]:
img_dir

'../../data/human-experiment/alignability-example/NeuEvo-diff_model-diff_epoch/set-15/img'

### Find Evolution

In [276]:
label_output_dir = '../../data/human-experiment/evolution-label-example'
ex_patch_dir = '../../data/neuron_feature'

In [273]:
num_samples_per_model = 25
models = [
    ['inception-v3-512-0.5-0.9', [3, 11, 121]],
    ['inception-v3-512-1.5-0.9', [4, 69, 71]],
    ['vgg16-512-0.01-0.9', [5, 21, 207]],
    ['vgg16-512-0.05-0.9', [3, 11, 13]]
]

In [286]:
for model_name, epochs in models:
    
    # Generate output path
    model_dir_path = os.path.join(label_output_dir, model_name)
    if not os.path.exists(model_dir_path):
        os.mkdir(model_dir_path)
    
    # Sample neurons
    all_neurons = list(emb[model_name][epochs[0]].keys())
    sampled_neurons = np.random.choice(all_neurons, num_samples_per_model, replace=False)
    
    # Generate data
    for i, sampled_neuron in enumerate(sampled_neurons):
        
        # Generate subdir path
        sub_dir_path = os.path.join(model_dir_path, 'set-{}'.format(i))
        if not os.path.exists(sub_dir_path):
            os.mkdir(sub_dir_path)
            
        # Write neuron info
        with open(os.path.join(sub_dir_path, 'neuron.json'), 'w') as f:
            neuron_info = {
                'model': model_name,
                'epochs': epochs,
                'neuron': sampled_neuron
            }
            json.dump(neuron_info, f, indent=4)
        
        # Save image
        for epoch_i, epoch in enumerate(epochs):
            
            # Directory to save images
            img_dir_path = os.path.join(
                sub_dir_path,
                'stage-{}'.format(epoch_i)
            )
            if not os.path.exists(img_dir_path):
                os.mkdir(img_dir_path)
                
            # Directory path for example patches
            model_code = '{}-{}'.format(model_name, epoch)
            ex_patch_dir_path = os.path.join(ex_patch_dir, model_code, 'data')
            ex_patch_dir_path = ex_patch_dir_path.replace('_', '-').replace('inception-v3', 'inception_v3')
            ex_patch_dir_path = ex_patch_dir_path.replace('neuron-feature', 'neuron_feature')
            
            # Image paths
            img_name_lst = []
            for img_name in os.listdir(ex_patch_dir_path):
                if sampled_neuron + '-' in img_name:
                    img_name_lst.append(img_name)

            img_path_lst = []
            for img_name in img_name_lst:
                img_path_lst.append(os.path.join(ex_patch_dir_path, img_name))

            # Save images
            for img_i, img_path in enumerate(img_path_lst):
                shutil.copy2(img_path, os.path.join(img_dir_path, 'img-{}.jpg'.format(img_i)))
        